In [42]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, transforms
from torchmetrics import Precision, Recall
import numpy as np
import pandas as pd
import os

In [31]:
np.random.seed(69)

In [28]:
# Data loader
def load_data(folders):
    """ load csvs as flattened feature vectors with labels """
    data, labels = [], []

    for folder, _ in folders:
        if os.path.exists(folder):
            for file in os.listdir(folder):
                if file.endswith('.csv'):
                    file_path = os.path.join(folder, file)
                    df = pd.read_csv(file_path)
                    label = df.iloc[0, 0]  # extract label
                    features = df.iloc[:, 1:].values.flatten()  # flatten features
                    data.append(features)
                    labels.append(label)
        else:
            print(f"warning: folder {folder} not found.")

    return np.array(data), np.array(labels)

In [29]:
# Define Training Folders
train_folders = [
    ('../rat_dance_csv/train', 1),  # label 1 for ratdance
    ('../neg_control_csv/train', 0)  # label 0 for negative control
]
# Load dataset
X, Y  = load_data(train_folders)


In [38]:
print(type(X))
print(type(Y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [27]:
class Nettik(nn.Module):
    def __init__ (self, num_classes):
        super().__init__()

        #Define the CNN for Image Classification
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, padding =1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten()     
        )
        self.classifier = nn.Linear(64*16*16, num_classes)
    
    def forward(self, x):
        x = self.model(x)
        x = self.classifier(x)
        return x

In [39]:
#Creating Training Loop for CNN 
net = Nettik(num_classes = 2)

#Criterion for Binary Classification; Cross entropy for multi-class
criterion = nn.BCELoss()

#Setting optiimzer for CNN
optimizer = optim.Adam(net.parameters(), lr = 0.001 )

#Creating the for-loop to execute training
for epoch in range(10):
    running_loss = 0.0
    for images, labels in train_folders: 
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss/len(train_folders)
    print(f"Epoch {epoch +1}, Loss: {epoch_loss:.4f}")

TypeError: conv2d() received an invalid combination of arguments - got (str, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)


In [45]:
#Creating the Evaluation Loop after training 
metric_precision = Precision(task="binary", num_classes=2, average="macro")
metric_recall = Recall(task="binary", num_classes=2, average="macro")

#Putting into eval mode
net.eval()

#Adding metrics for recall and precision
with torch.no_grad():
    for images, labels in train_folders:
        outputs = net(images)
        _, preds = torch.max(outputs, 1)
        metric_precision(preds, labels)
        metric_recall(preds, labels)
precision = metric_precision.compute()
recall = metric_recall.compute()

print(f"Precision: {precision}")
print(f"Recall: {recall} ")

TypeError: conv2d() received an invalid combination of arguments - got (str, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!str!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
